In [1]:
# Data Import and Pre-Processing
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [44]:
# train_words_raw = pd.read_csv('./data/train', sep='\t', header=None, names=['word', 'label'])
# train_vocab = set(train_words_raw.word.unique())

In [45]:
def load_sequences_from_words(filename, sep="\t"):
    sequences = []
    with open(filename) as fp:
        seq = []
        for line in fp:
            line = line.strip()
            if line:
                line = line.split(sep)
                seq.append(line)
            else:
                sequences.append(seq)
                seq = []
        if seq:
            sequences.append(seq)
    return sequences

In [83]:
#train_seq = load_sequences_from_words('./data/train')
#train_vocab

In [84]:
def load_vocab(filename):
    vocab = set()
    with open(filename) as fp:
        for line in fp:
            line = line.split('\t')
            vocab.add(line[0])
    return vocab 

In [85]:
train_vocab = load_vocab('./data/train')

In [ ]:
# Model Start

In [125]:
import tensorflow as tf

tf.reset_default_graph()

In [126]:
# Model Params

learning_rate = 0.01
train_epoch = 10000
input_size = 10
batch_size = 100
num_units = 512
num_classes = 5

In [127]:
# Placeholders

inputs = tf.placeholder(tf.float32, [None, None, input_size], name='inputs')
labels = tf.placeholder(tf.int32, [None, None], name='labels')
batch_seq_len = tf.placeholder(tf.int32)
org_seq_len = tf.placeholder(tf.int32, [None])

In [128]:
# Bi-LSTM Cell
with tf.name_scope("BiLSTM"):
    with tf.variable_scope('forward'):
        lstm_fw = tf.nn.rnn_cell.LSTMCell(num_units, forget_bias=1.0, state_is_tuple=True)
    with tf.variable_scope('backward'):
        lstm_bw = tf.nn.rnn_cell.LSTMCell(num_units, forget_bias=1.0, state_is_tuple=True)
    
    (output_fw, output_bw), states = tf.nn.bidirectional_dynamic_rnn(cell_fw=lstm_fw,
                                                                     cell_bw=lstm_bw,
                                                                     inputs=inputs,
                                                                     sequence_length=org_seq_len,
                                                                     dtype=tf.float32,
                                                                     scope="BiLSTM")



outputs = tf.concat([output_fw, output_bw], axis=2)

# FC
W = tf.get_variable("W", [2 * num_units, num_classes], dtype=tf.float32)

b = tf.get_variable("b", [num_classes], dtype=tf.float32, initializer=tf.zeros_initializer())

outputs_flat = tf.reshape(outputs, [-1, 2 * num_units])
pred = tf.matmul(outputs_flat, W) + b
scores = tf.reshape(pred, [-1, batch_seq_len, num_classes])

# CRF

log_loss, trans_params = tf.contrib.crf.crf_log_likelihood(scores, labels, org_seq_len)
loss = tf.reduce_mean(-log_loss)

# viterbi Seq, score
viterbi_seq, viterbi_score = tf.contrib.crf.crf_decode(scores, trans_params, org_seq_len)

# Train Ops
train_opt = tf.train.AdamOptimizer(learning_rate)
train_op = train_opt.minimize(loss)

# Saver
saver = tf.train.Saver()

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
# Train into Session

# Take From Batch
batch_inputs = []
batch_labels = []
batch_seq_lengths = 0

with tf.Sesion() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(train_epoch):
        for batch_inputs, batch_labels, batch_len, batch_seq_lengths in batch(x_, y_, seq_len_train, batch_size, input_size):
            tf_viterbi_seq, _ = sess.run([viterbi_seq, train_op],
                                        feed_dict={inputs: batch_inputs,
                                                  labels: batch_labels,
                                                  batch_seq_len:batch_len,
                                                  org_seq_len: batch_seq_lengths})
        
            if i % 30 == 0:
                mask = (np.expand_dims(np.arrange(batch_len), axis=0) < np.expand_dims(batch_seq_lengths, axis=1))
                total_labels = np.sum(batch_seq_lengths)
                correct_labels = np.sum((batch_labels == tf_viterbi_seq)* mask)
                accuracy = 100.0 * correct_labels / float(total_labels)
                print("Epoch ", i, " Accuracy: ", accuracy)
        
    saver.save(sess, './model_crf')

In [ ]:
# Test Accuracy and Pred

# Take From Batch
batch_test = []
batch_test_labels = []
batch_seq_t_lengths = 0

with tf.Sesion() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(train_epoch):
        for batch_inputs, batch_labels, batch_len, batch_seq_t_lengths in batch(x_t, y_t, seq_len_test, batch_test_size, input_size):
            tf_viterbi_seq = sess.run(viterbi_seq,
                                        feed_dict={inputs: batch_inputs,
                                                  labels: batch_labels,
                                                  batch_seq_len:batch_len,
                                                  org_seq_len: batch_seq_t_lengths})
        
            
            mask = (np.expand_dims(np.arrange(batch_len), axis=0) < np.expand_dims(batch_seq_lengths, axis=1))
            total_labels = np.sum(batch_seq_t_lengths)
            correct_labels = np.sum((batch_labels == tf_viterbi_seq)* mask)
            accuracy = 100.0 * correct_labels / float(total_labels)
            print("Test Accuracy: ", accuracy)
            print("Label: ", batch_labels[0].astype(int))
            print("Pred: ", tf_viterbi_sequence[0])